##### 웃긴대학
- URL : http://web.humoruniv.com/main.html
- 인기자료(게시판): http://web.humoruniv.com/board/humor/list.html?table=pick

##### 전체코드

In [ ]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import time
from urllib.request import urlopen, Request
from urllib.parse import urlparse, urljoin, urlencode, unquote, quote, quote_plus
import requests
import re


def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp



from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from bs4 import BeautifulSoup

# DB 파일 생성
conn = sqlite3.connect('humoruniv.db')
cur = conn.cursor()
# DROP TABLE IF EXISTS humoruniv;
# cur.executescript('''
#     CREATE TABLE humoruniv(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         url TEXT NOT NULL,
#         head TEXT NOT NULL,
#         body TEXT NOT NULL,
#         comment TEXT NOT NULL,
#         date TEXT NOT NULL
#     );
# ''')
conn.commit()

# url, params, header 정의
url = 'http://web.humoruniv.com/board/humor/list.html?'
params = {
    'table': 'pick',
    'pg': 0
}
headers = {
    'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"
}

# ---------- 첫 글의 url을 가려와서 이전 글 url을 계속 타고가는 방법 -----------

resp1 = download(url, params, headers, 'GET')
dom1 = BeautifulSoup(resp1.content, 'lxml')

time.sleep(0.02)

# 첫 글의 url 받아오기
newurl = urljoin(url, dom1.select_one('#cnts_list_new td[class=li_sbj] a')['href'])
print(newurl)

# 개수 확인을 위한 count
count = -1

# 이전 글 url이 없을 때까지 반복하며 데이터 추출 및 수집
while True:
        resp2 = download(newurl, params, headers, 'GET')
        dom2 = BeautifulSoup(resp2.content, 'html.parser')
        
        time.sleep(0.02)
        
        # 수집 개수, 페이지 확인
        count += 1
        print(count)
        #print('page',int(count/20))
        
        # url 확인 출력
        #print(newurl)
        
        # 원하는 데이터 추출
        head = dom2.select_one('#ai_cm_title').text.strip()
        #print('head', head)
        body = dom2.select_one('#wrap_body #ai_cm_content').text.strip()
        #print('body', body)
        comments = '\n'.join(_.text.strip() for _ in dom2.select('#list_best_box_text span[class=cmt_list]'))
        #print('comments', comments)
        date= re.search(r'(\d+-\d+-\d+\s\d+:\d+:\d+)', dom2.select_one('#if_date').text)[1]
        #print('date', date)
        
        # DB에 INSERT
        cur.execute('INSERT INTO humoruniv(url, head, body, comment, date) VALUES(?,?,?,?,?)', [newurl, head, body, comments, date])
        conn.commit()
        
        # 이전글 url
        newhref = dom2.select_one('div[class=prv] tr:nth-of-type(2) a')['href']

        # 이전글 url이 없다면 while문 탈출
        if newhref == []:
                break

        # 이전글 url을 로 새로운 url 생성
        newurl = urljoin(newurl, newhref)
        

##### 인터넷 연결 문제나 호스트 강제종료 시 이어서 실행

In [148]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import time
from urllib.request import urlopen, Request
from urllib.parse import urlparse, urljoin, urlencode, unquote, quote, quote_plus
import requests
import re


def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp



from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from bs4 import BeautifulSoup

# DB
conn = sqlite3.connect('humoruniv.db')
cur = conn.cursor()

# url 설정
params ={
}
headers = {
    'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"
}
url = 'http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955799&device=pc'
#print(newurl)


resp1 = download(url, params, headers, 'GET')
dom1 = BeautifulSoup(resp1.content, 'html.parser')


time.sleep(0.02)


newhref = dom1.select_one('div[class=prv] tr:nth-of-type(2) a')['href']
newurl = urljoin(url, newhref)


count = 12066

# 이전 글 url이 없을 때까지 반복하며 데이터 추출 및 수집
while True:
        resp2 = download(newurl, params, headers, 'GET')
        dom2 = BeautifulSoup(resp2.content, 'html.parser')
                
        time.sleep(0.02)
        
        # url 확인 출력
        print(newurl)

        # 수집 개수, 페이지 확인
        count += 1
        print(count)
        #print('page',int(count/20))

        
        # 원하는 데이터 추출
        head = dom2.select_one('#ai_cm_title').text.strip()
        #print('head', head)
        body = dom2.select_one('#wrap_body #ai_cm_content').text.strip()
        #print('body', body)
        comments = '\n'.join(_.text.strip() for _ in dom2.select('#list_best_box_text span[class=cmt_list]'))
        #print('comments', comments)
        date= re.search(r'(\d+-\d+-\d+\s\d+:\d+:\d+)', dom2.select_one('#if_date').text)[1]
        #print('date', date)
        
        # DB에 INSERT
        cur.execute('INSERT INTO humoruniv(url, head, body, comment, date) VALUES(?,?,?,?,?)', [newurl, head, body, comments, date])
        conn.commit()
        
        # 이전글 url
        newhref = dom2.select_one('div[class=prv] tr:nth-of-type(2) a')['href']

        # 이전글 url이 없다면 while문 탈출
        if newhref == []:
                break

        # 이전글 url을 로 새로운 url 생성
        newurl = urljoin(newurl, newhref)

http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955798&device=pc
12067
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955795&device=pc
12068
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955794&device=pc
12069
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955790&device=pc
12070
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955787&device=pc
12071
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955786&device=pc
12072
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955785&device=pc
12073
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955784&device=pc
12074
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955782&device=pc
12075
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955779&device=pc
12076
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=

http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955586&device=pc
12156
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955580&device=pc
12157
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955576&device=pc
12158
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955574&device=pc
12159
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955573&device=pc
12160
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955572&device=pc
12161
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955570&device=pc
12162
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955568&device=pc
12163
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955564&device=pc
12164
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955563&device=pc
12165
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=

http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955398&device=pc
12245
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955396&device=pc
12246
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955393&device=pc
12247
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955391&device=pc
12248
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955389&device=pc
12249
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955388&device=pc
12250
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955386&device=pc
12251
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955375&device=pc
12252
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955374&device=pc
12253
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955368&device=pc
12254
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=

http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955213&device=pc
12334
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955212&device=pc
12335
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955211&device=pc
12336
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955206&device=pc
12337
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955203&device=pc
12338
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955202&device=pc
12339
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955201&device=pc
12340
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955200&device=pc
12341
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955199&device=pc
12342
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955197&device=pc
12343
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955088&device=pc
12388
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955078&device=pc
12389
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955075&device=pc
12390
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955069&device=pc
12391
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955068&device=pc
12392
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955067&device=pc
12393
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955064&device=pc
12394
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955062&device=pc
12395
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955054&device=pc
12396
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955052&device=pc
12397
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955040&device=pc
12399
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955038&device=pc
12400
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955037&device=pc
12401
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955034&device=pc
12402
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955031&device=pc
12403
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955029&device=pc
12404
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955028&device=pc
12405
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955027&device=pc
12406
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955025&device=pc
12407
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=955023&device=pc
12408
http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=

AttributeError: 'NoneType' object has no attribute 'text'

----------------------------------
##### 아래와 같은 오류 종종 발생

---------------------
##### 해결 방법 못찾음 . (실패 >> 마지막 url부터 다시실행)

AttributeError                            Traceback (most recent call last)
<ipython-input-15-fbff7afef494> in <module>
     52 
     53         # 원하는 데이터 추출
---> 54         head = dom2.select_one('#ai_cm_title').text.strip()
     55         #print('head', head)
     56         body = dom2.select_one('#wrap_body #ai_cm_content').text.strip()

AttributeError: 'NoneType' object has no attribute 'text'

11942
    
---------------------
##### time 추가

4657 ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다

8521 ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다

9208 TypeError: 'NoneType' object is not subscriptable
9208 http://web.humoruniv.com/board/humor/read.html?table=pick&pg=0&number=962442&device=pc
시스템 과부하 혹은 지나친 접속이 감지되어 서비스가 지연되고 있습니다. 잠시 기다리신 후에 다시 시도해 주세요. [2020-07-27 09:50:27]

In [ ]:
# from requests import request
# from requests.compat import urljoin, urlparse
# from requests.exceptions import HTTPError
# from urllib.robotparser import RobotFileParser
# from bs4 import BeautifulSoup
# from time import sleep
# import time
# from urllib.request import urlopen, Request
# from urllib.parse import urlparse, urljoin, urlencode, unquote, quote, quote_plus
# import requests
# import re

In [ ]:
# def canfetch(url, agent='*', path='/'):
#     robot = RobotFileParser(urljoin(url, '/robots.txt'))
#     robot.read()
#     return robot.can_fetch(agent, urlparse(url)[2])
    
# def download(url, params={}, headers={}, method='GET', limit=3):
#     if canfetch(url) == False:
#         print('[Error] ' + url)
# #     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
#     try:
#         resp = request(method, url,
#                params=params if method=='GET' else {},
#                data=params if method=='POST' else {},
#                headers=headers)
#         resp.raise_for_status()
#     except HTTPError as e:
#         if limit > 0 and e.response.status_code >= 500:
#             print(limit)
#             time.sleep(1) # => random
#             resp = download(url, params, headers, method, limit-1)
#         else:
#             print('[{}] '.format(e.response.status_code) + url)
#             print(e.response.status_code)
#             print(e.response.reason)
#             print(e.response.headers)
#     return resp

In [ ]:
# url = 'http://web.humoruniv.com/board/humor/list.html'
# params = {
#     'table': 'pick',
#     'pg':0
# }
# headers = {
#     'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"
# }

# resp1 = download(url, params, headers, 'GET')
# dom1 = BeautifulSoup(resp1.content, 'html.parser')